In [1]:
import pandas as pd
import requests

from bs4 import BeautifulSoup
import re

from nltk.stem import WordNetLemmatizer, PorterStemmer, SnowballStemmer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter


In [2]:
def preprocess(raw_list):
    cleaned_text_list = []
    
    stop_words_file = 'english.txt'
    stop_words = []

    with open(stop_words_file, "r") as f:
        for line in f:
            stop_words.extend(line.split()) 
    for raw_text in raw_list:
        stop_words = stop_words  
        letters_only_text = re.sub("[^a-zA-Z]", " ", raw_text)

        words = letters_only_text.lower().split()

        cleaned_words = []

        for word in words:
            if word not in stop_words:
                cleaned_words.append(word)
        cleaned_text = " ".join(cleaned_words)
        cleaned_text_list.append(cleaned_text)

    return cleaned_text_list

In [7]:
def fwords(list_text, df_raw):

    dict_words = {}

    for i, text in enumerate(list_text):
        tokens = word_tokenize(text)
        stop_words = set(stopwords.words("english"))
        words = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]
        word_freq = Counter(words)
        sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
        
        dict_word = {}
        for word, freq in sorted_words[:5]:
            dict_word[word] = freq
        dict_words[i] = dict_word
    df_words=pd.DataFrame(dict_words)

    df_words= df_words.transpose()
    
    sums = df_words.sum(axis=0).sort_values(ascending=False)
    df_words = df_words[sums.index]
    
    df_words= df_words.iloc[:, :20]
    df_together=pd.merge(df_raw,df_words, left_index=True,  right_index=True)
    return df_together

In [8]:
df_bbc_china=pd.read_csv('data/bbc_china.csv')

In [9]:
text_cleaned=preprocess(df_bbc_china['article'])


In [10]:
df_together=fwords(text_cleaned,df_bbc_china)

In [11]:
df_together

,title,article,link,day,month,year,china,covid,balloon,chinese,...,south,beijing,taiwan,cases,xi,military,war,philippines,africa,government
0,South China Sea: Philippines says China used '...,South China Sea: Philippines says China used ´...,https://www.bbc.co.uk/news/world-asia-64621414,13,2,2023,12.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN
1,China says US balloons breached airspace at le...,China says US balloons breached airspace at le...,https://www.bbc.co.uk/news/world-asia-china-64...,13,2,2023,8.0,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,China Covid: Five deaths under country's new c...,China Covid: Five deaths under country´s new c...,https://www.bbc.co.uk/news/world-asia-64044204,20,12,2022,11.0,15.0,NaN,NaN,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN
3,India Covid: Experts say people don't need to ...,India Covid: Experts say people don´t need to ...,https://www.bbc.co.uk/news/world-asia-india-64...,28,12,2022,12.0,15.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,China ends Covid quarantine for travellers in ...,China ends Covid quarantine for travellers in ...,https://www.bbc.co.uk/news/world-asia-china-64...,27,12,2022,13.0,17.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,China: Five dead after man drives into crowd i...,China: Five dead after man drives into crowd i...,https://www.bbc.co.uk/news/world-asia-china-64...,12,1,2023,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,China suspends social media accounts of Covid ...,China suspends social media accounts of Covid ...,https://www.bbc.co.uk/news/world-asia-china-64...,7,1,2023,8.0,11.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,China Covid: Hospitals under strain in wave of...,China Covid: Hospitals under strain in wave of...,https://www.bbc.co.uk/news/world-asia-china-63...,14,12,2022,12.0,16.0,NaN,NaN,...,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
126,Liz Truss: Ex-PM to join global campaign to pu...,Liz Truss: Ex-PM to join global campaign to pu...,https://www.bbc.co.uk/news/uk-politics-64509979,3,2,2023,7.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
